In [1]:
import os
import cv2
import numpy as np

# Define the parent directory
parent_dir = 'chapters_pp'

# Get a list of all the subdirectories in the parent directory
subdirs = [os.path.join(parent_dir, subdir) for subdir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, subdir))]

# Initialize an empty list to hold the images
images = []

# Loop over the subdirectories
for subdir in subdirs:
    # Get a list of all the image files in the current subdirectory
    image_files = [os.path.join(subdir, file) for file in os.listdir(subdir) if file.endswith('.jpeg')]
    
    # Loop over the image files
    for image_file in image_files:
        # Load the current image file
        image = cv2.imread(image_file)
        
        # Resize the image to 256x256
        image = cv2.resize(image, (256, 256))
        
        # Normalize the image to [0, 1]
        image = image / 255.0
        
        # Add the image to the list of images
        images.append(image)

# Convert the list of images to a numpy array
images = np.array(images)

In [4]:
from keras.layers import Input, Dense, Lambda, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.losses import binary_crossentropy

# Define the encoder
input_img = Input(shape=(256, 256, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Define the decoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Define the VAE
vae = Model(input_img, decoded)

# Define the VAE loss
def vae_loss(input_img, decoded):
    reconstruction_loss = binary_crossentropy(K.flatten(input_img), K.flatten(decoded))
    reconstruction_loss *= 256 * 256
    return reconstruction_loss

# Compile the VAE
# Resize the images to a smaller dimension
resized_images = []
for img in images:
    resized_img = cv2.resize(img, (128, 128))
    resized_images.append(resized_img)
resized_images = np.array(resized_images, dtype=np.float32)  # Change the data type to float32


MemoryError: Unable to allocate 1.48 GiB for an array with shape (4044, 128, 128, 3) and data type float64

In [ ]:
# Use a Variational Autoencoder to generate new images of preprocessed images
# from the Chapters_PP folder

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Lambda
from keras import backend as K

# Load the model
model = load_model('vae.h5')

# Load the preprocessed jpeg images for every chapter subfolder
chapters = os.listdir('Chapters_PP')
images = []
for chapter in chapters:
    for img in os.listdir('Chapters_PP/' + chapter):
        img = image.load_img('Chapters_PP/' + chapter + '/' + img, target_size=(256, 256))
        img = image.img_to_array(img)
        images.append(img)
images = np.array(images)
images = images.astype('float32') / 255

# Encode the images
encoder = Model(input=model.input, output=model.get_layer('z_mean').output)
encoded_images = encoder.predict(images)

# Generate new images
decoder_input = Input(shape=(2,))
decoder = model.layers[-1]
decoder_output = decoder(decoder_input)
decoder = Model(decoder_input, decoder_output)
generated_images = decoder.predict(encoded_images)

# Plot the generated images
n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(images[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(generated_images[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Save the generated images
for i in range(n):
    img = image.array_to_img(generated_images[i])
    img.save('generated_image_' + str(i) + '.jpg')

# Save the encoded images
np.save('encoded_images.npy', encoded_images)

# Save the decoded images in a dedicated folder for each chapter
for i, chapter in enumerate(chapters):
    os.makedirs('Chapters_PP_Decoded/' + chapter, exist_ok=True)
    for j in range(len(images)):
        img = image.array_to_img(generated_images[j])
        img.save('Chapters_PP_Decoded/' + chapter + '/generated_image_' + str(j) + '.jpeg')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'vae.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)